In [1]:
import tools

In [4]:
gpx_filename = 'data-input/28may2021.gpx'

In [7]:
import geopandas as gpd
from shapely.geometry import LineString, Point

data_activity = tools.getActivityDataFromGpxFile(gpx_filename)
points_full = data_activity['tracks'][0]['segments'][0]['points']
 
points = [point[:2] for point in points_full]
gdf = gpd.GeoDataFrame(geometry=[LineString(points)])

In [10]:
from ipyleaflet import Map, Polyline
from ipywidgets import Layout

In [11]:
gdf1 = tools.simplify(gdf,  tolerance=0.0000001)
gdf2 = tools.simplify(gdf1, tolerance=0.000001)
gdf3 = tools.simplify(gdf2, tolerance=0.00001)
gdf4 = tools.simplify(gdf3, tolerance=0.0001)
gdf5 = tools.simplify(gdf4, tolerance=0.0001)
gdf9 = tools.simplify(gdf5, tolerance=0.01)

gdf_spline = tools.gdf2splines(gdf9)


m = Map(center=tools.gdf2centroid(gdf9), zoom=13, layout=Layout(width='100%', height='700px'))

m.add(tools.gdf2line(gdf1, 'navy'))
m.add(tools.gdf2line(gdf5, 'yellow'))

m.add(tools.gdf2line(gdf9, 'red'))

m.add(tools.gdf2line(gdf_spline, 'green'))
m


Simplify Coords count:  11589  ->  10969
Tolerance:  1e-07

Simplify Coords count:  10969  ->  7676
Tolerance:  1e-06

Simplify Coords count:  7676  ->  1977
Tolerance:  1e-05

Simplify Coords count:  1977  ->  386
Tolerance:  0.0001

Simplify Coords count:  386  ->  379
Tolerance:  0.0001

Simplify Coords count:  379  ->  13
Tolerance:  0.01



Map(center=[55.67781692843676, 36.71038865754297], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [ ]:
from datetime import datetime

tools.gdf_reverse_coords(gdf5).to_file(f'data-output/zvenigorod-route-{int(datetime.now().timestamp())}.geojson', driver='GeoJSON')


In [28]:
camera_points = [
    (55.563636, 36.592460),
    (55.575730, 36.530853),
    (55.608732, 36.548689),
    (55.634572, 36.640942),
    (55.707225, 36.652690),
    (55.705424, 36.714075),
    (55.705150, 36.807214),
    (55.692771, 36.824143),
    (55.701127, 36.932805),
]

gdf_camera_spliens = tools.gdf2splines(tools.points2gdf(camera_points))

m.add(tools.gdf2line(gdf_camera_spliens, 'magenta'))
m

Map(bottom=41872.0, center=[54.89872361296002, 38.08410644531251], controls=(ZoomControl(options=['position', …

In [29]:
import string
import utils
from datetime import datetime

# Make json

js_file_template = string.Template('''
const routes = {
    target: [
    $target
],
    camera: [
    $camera
    ]
};
''')


text = js_file_template.substitute(
    target = ',\n'.join([f'\t\t[{c[0]}, {c[1]}]' for c in gdf2coords(gdf_reverse_coords(gdf_spline2))]),
    camera = ',\n'.join([f'\t\t[{c[0]}, {c[1]}]' for c in gdf2coords(gdf_reverse_coords(gdf_camera_spliens))])
)

utils.write_file(f'data-output/zvenigorod-camera-{int(datetime.now().timestamp())}.js', text)


PosixPath('data-output/zvenigorod-camera-1712171569.js')

In [67]:

export_js('mskappr', 'camera', camera_points)

🚀 Export. Path and Variable: 
maps/notebook/data-output/mskappr-camera.js
cameraCoordinates


In [ ]:
import math

st_cam_points = [
    (55.577952, 36.515800),\
    (55.690757, 36.825998),
]
print(st_cam_points)
print()

dgf_st_cam = gpd.GeoDataFrame({'geometry': [LineString(st_cam_points)]})
print(dgf_st_cam)


delta = 0.02

st_target_points = [(round(p[0]+delta, 6), round(p[1]-delta, 6))for p in st_cam_points]
print(st_target_points)
print()

dgf_st_target = gpd.GeoDataFrame({'geometry': [LineString(st_target_points)]})
print(dgf_st_target)


m_st = Map(center=gdf2centroid(gdf9), zoom=11)
m_st.add(gdf2line(dgf_st_cam, 'cyan'))
m_st.add(gdf2line(dgf_st_target, 'lime'))
m_st



In [42]:


text = js_file_template.substitute(
    target = ',\n'.join([f'\t\t[{c[0]}, {c[1]}]' for c in gdf2coords(gdf_reverse_coords(dgf_st_target))]),
    camera = ',\n'.join([f'\t\t[{c[0]}, {c[1]}]' for c in gdf2coords(gdf_reverse_coords(dgf_st_cam))])
)

utils.write_file(f'data-output/st-line-{int(datetime.now().timestamp())}.js', text)

PosixPath('data-output/st-line-1712168373.js')

In [15]:
red_coords = gdf2coords(gdf9)
print(red_coords)

[(55.585876, 36.566525), (55.584892, 36.570239), (55.599562, 36.600785), (55.63986, 36.582073), (55.646723, 36.635482), (55.675792, 36.688909), (55.703134, 36.679394), (55.707225, 36.75802), (55.722355, 36.761598), (55.708746, 36.801406), (55.730646, 36.840554), (55.700564, 36.881551), (55.692896, 36.87081)]


In [24]:
centers = []

for idx, p in enumerate(red_coords):
    if idx == 0:
        continue

    c0 = (red_coords[idx-1][0] + red_coords[idx][0] )/2
    c1 = (red_coords[idx-1][1] + red_coords[idx][1] )/2
    centers.append((round(c0, 6), round(c1, 6)))

print(centers)

gdf_spline2 = gdf2splines(points2gdf(centers))

m_center = Map(center=gdf2centroid(gdf9), zoom=11)

m_center.add(gdf2line(gdf9, 'red'))
m_center.add(gdf2line(gdf_spline2, 'orange'))
m_center

[(55.585384, 36.568382), (55.592227, 36.585512), (55.619711, 36.591429), (55.643292, 36.608778), (55.661258, 36.662196), (55.689463, 36.684151), (55.705179, 36.718707), (55.71479, 36.759809), (55.71555, 36.781502), (55.719696, 36.82098), (55.715605, 36.861052), (55.69673, 36.876181)]


Map(center=[55.67781692843676, 36.71038865754297], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [ ]:
# altitide



alts = [
    
]

In [ ]:
import numpy as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

# Generate some sample data points
x = np.array([0, 1, 2, 3, 4, 5])

y = np.array([0, 1, 4, 9, 16, 25])

# Create a cubic spline interpolation function
cs = CubicSpline(x, y)

# Generate points for plotting the spline
x_interp = np.linspace(0, 5, 100)
y_interp = cs(x_interp)

# Plot original data points and the cubic spline
plt.scatter(x, y, label='Data Points')
plt.plot(x_interp, y_interp, label='Cubic Spline', color='red')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Cubic Spline Interpolation')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print(1+2)